#### 导入库

In [1]:
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
import chromadb
import os

#### 文档加载&切分&检索

In [2]:
! python ingest.py

python: can't open file '/home/linjw/AGI/llama2/langchain/ingest.py': [Errno 2] No such file or directory


In [3]:
import os
from dotenv import load_dotenv
from chromadb.config import Settings

load_dotenv()

# Define the folder for storing database
PERSIST_DIRECTORY = "/home/linjw/AGI/llama2/langchain/"
# Define the Chroma settings
CHROMA_SETTINGS = Settings(
        persist_directory=PERSIST_DIRECTORY,
        anonymized_telemetry=False
)

embeddings_model_name = "/home/linjw/AGI/Weight/paraphrase-multilingual-MiniLM-L12-v2"
persist_directory = "/home/linjw/AGI/privateGPT/db"
model_path = os.environ.get('MODEL_PATH')
model_n_ctx = os.environ.get('MODEL_N_CTX')
model_n_batch = int(os.environ.get('MODEL_N_BATCH',8))
target_source_chunks = int(os.environ.get('TARGET_SOURCE_CHUNKS',4))
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
chroma_client = chromadb.PersistentClient(settings=CHROMA_SETTINGS , path=persist_directory)
db = Chroma(persist_directory=persist_directory, embedding_function=embeddings, client_settings=CHROMA_SETTINGS, client=chroma_client)

##### 普通检索

In [4]:
retriever = db.as_retriever(search_kwargs={"k": 100})

# retriever = db.as_retriever(    
#     search_type="mmr",
#     search_kwargs={'k': 20, 'lambda_mult': 0.25})


    参数：

    search_type（可选）：定义检索器应该执行的搜索类型。可以是 "similarity"（默认）、"mmr" 或 "similarity_score_threshold"。
    search_kwargs（可选）：传递给搜索函数的关键字参数，可以包括诸如以下内容：
    k：要返回的文档数量（默认值为 4）。
    score_threshold：用于 "similarity_score_threshold" 的最小相关性阈值。
    fetch_k：传递给 MMR 算法考虑的文档数量（默认值为 20）。
    lambda_mult：MMR 返回的结果的多样性；1 表示最小多样性，0 表示最大多样性（默认值为 0.5）。
    filter：根据文档元数据进行筛选。

In [5]:
query = "AI医疗概念是什么"
docs = retriever.get_relevant_documents(query)
[doc.page_content for doc in docs]

['AI 赋能医疗的发展是向多主体提供智慧服务，面向医疗机构的智慧医院建设，涉及患者、医疗(包\n括门诊、住院)、护理、医技(含药事)、管理(含行政、业务)、后勤保障、教学科研、区域协调等领\n域的智慧化建设，是一个系统性的工程。面向监管机构的智慧监管建设，涉及医疗数据、医疗行\n为、医疗费用、医疗人事等方面的监管，AI 需要助力实现医疗数据的隐私保护和权限分配，医疗行\n为的科学性和合规性，医疗费用的合理性和真实性以及医疗人事组织的灵活性。 \n面向产业生态的智慧服务，为医药企业提供靶点发现与探索、化合物筛选与优化、临床前研究、临\n床研究、注册申报、真实世界研究服务，助力器械企业研发医疗 AI 设备，为互联网医疗企业提供\n智能问诊、智能续方、智能患者管理服务，为保险企业提供智能分销、智能定价、智能理赔服务，\n为药店提供智能采购、承接处方、患者管理服务，为第三方医检企业提供影像、病理辅助诊断服务\n等。面向患者的智慧管理建设，包括健康管理、在线复诊、慢病管理、康复护理、在线购药等服\n务。',
 '五个领域展开。 \nAI 医学影像是指将人工智能技术应用在医学影像的诊断上。AI 医学影像被\n业内人认为是最有可能率先实现商业化的人工智能医疗领域，当前很多落地的应\n用也主要以 AI 医学影像方向。 \nAI 辅助诊断主要提供医学影像、电子病历、导诊机器人、虚拟助理等服务，\n利用机器学习+计算机视觉缓解病理专家稀缺、医生素质不高的现状，利用人工\n智能+大数据对患者进行系统化记录和健康管理，利用人工智能+机器人技术分担\n医院从医人数不足的压力。 \nAI 新药研发主要包括药物发现、临床前研究、临床研究 以及审批与上市四\n个阶段。利用机器学习和人工智能可以减少传统药物研发的时间成本与精力成本，\n中国人工智能产业发展联盟AI I A',
 '5\n1.1 行业定义\n医疗AI以助力降本增效、提升诊疗水平等为核\n心目的，全面赋能院前、院中、院后各个环节\n•\n医疗AI是指将计算机视觉、语音识别、自然语言处理、机器学习等人工智能技\n术应用于医疗领域中。近年来，随着人工智能技术的加速成熟，其在医疗健康\n领域的应用场景不断丰富，为疾病检测、诊断及治疗模式带来深刻变革，为提\n升居民健康质量提供新方式。\n•\n目前，人工智能技术在医疗领域的应用主要包括医学影

In [6]:
for doc in docs:
    print(doc.metadata)


{'author': '', 'creationDate': '', 'creator': '', 'file_path': 'source_documents/2021年  【88页】AI新药研发（AIDD）行业系列报告：洞鉴行业发展，把握投资先机：（一）AIDD概览篇-AI新药研发已走过0到1阶段，1到10阶段将带来更多可能.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20210903094738+08'00'", 'page': 13, 'producer': 'Microsoft® Word 2010; modified using iText® 5.4.2 ©2000-2012 1T3XT BVBA (AGPL-version); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': 'source_documents/2021年  【88页】AI新药研发（AIDD）行业系列报告：洞鉴行业发展，把握投资先机：（一）AIDD概览篇-AI新药研发已走过0到1阶段，1到10阶段将带来更多可能.pdf', 'subject': '', 'title': '中银证券-医药生物行业AI新药研发(AIDD)行业系列报告-洞鉴行业发展，把握投资先机：(一)AIDD概览篇，AI新药研发已走过0到1阶段，1到10阶段将带来更多可能-210902.pdf', 'total_pages': 88, 'trapped': ''}
{'author': '', 'creationDate': "D:20210202143037+08'00'", 'creator': '', 'file_path': 'source_documents/2020年  【443页】中国人工智能产业知识产权白皮书（2020）.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20210205151303+08'00'", 'page': 97, 'producer': '福昕PDF编辑器打印机版本10.0.221.3451; modified using iText® 5.5.13.1 ©2000-2019 i

##### LLM辅助检索

##### LLM压缩检索

In [7]:
# from langchain.retrievers import ContextualCompressionRetriever
# from langchain.retrievers.document_compressors import LLMChainExtractor
# from langchain.llms import OpenAI
# from langchain.retrievers.self_query.base import SelfQueryRetriever
# from langchain.chains.query_constructor.base import AttributeInfo
 
# #定义元数据的过滤条件
# metadata_field_info = [
#     AttributeInfo(
#         name="source",
#         description="The lecture the chunk is from, should be one of `docs/cs229_lectures/MachineLearning-Lecture01.pdf`, `docs/cs229_lectures/MachineLearning-Lecture02.pdf`, or `docs/cs229_lectures/MachineLearning-Lecture03.pdf`",
#         type="string",
#     ),
#     AttributeInfo(
#         name="page",
#         description="The page from the lecture",
#         type="integer",
#     ),
# ]
 
# #创建SelfQueryRetriever
# document_content_description = "Lecture notes"
# llm = OpenAI(temperature=0)
# retriever = SelfQueryRetriever.from_llm(
#     llm,
#     db,
#     document_content_description,
#     metadata_field_info,
#     verbose=True
# )
# def pretty_print_docs(docs):
#     print(f"\n{'-' * 100}\n".join([f"Document {i+1}:\n\n" + d.page_content for i, d in enumerate(docs)]))
    
# # Wrap our vectorstore
# llm = OpenAI(temperature=0)
# compressor = LLMChainExtractor.from_llm(llm)
 
# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=compressor,
#     base_retriever=db.as_retriever()
# )

# question = "what did they say about matlab?"
# compressed_docs = compression_retriever.get_relevant_documents(question)
# pretty_print_docs(compressed_docs)

#### 加载模型

In [8]:
docs

[Document(page_content='AI 赋能医疗的发展是向多主体提供智慧服务，面向医疗机构的智慧医院建设，涉及患者、医疗(包\n括门诊、住院)、护理、医技(含药事)、管理(含行政、业务)、后勤保障、教学科研、区域协调等领\n域的智慧化建设，是一个系统性的工程。面向监管机构的智慧监管建设，涉及医疗数据、医疗行\n为、医疗费用、医疗人事等方面的监管，AI 需要助力实现医疗数据的隐私保护和权限分配，医疗行\n为的科学性和合规性，医疗费用的合理性和真实性以及医疗人事组织的灵活性。 \n面向产业生态的智慧服务，为医药企业提供靶点发现与探索、化合物筛选与优化、临床前研究、临\n床研究、注册申报、真实世界研究服务，助力器械企业研发医疗 AI 设备，为互联网医疗企业提供\n智能问诊、智能续方、智能患者管理服务，为保险企业提供智能分销、智能定价、智能理赔服务，\n为药店提供智能采购、承接处方、患者管理服务，为第三方医检企业提供影像、病理辅助诊断服务\n等。面向患者的智慧管理建设，包括健康管理、在线复诊、慢病管理、康复护理、在线购药等服\n务。', metadata={'author': '', 'creationDate': '', 'creator': '', 'file_path': 'source_documents/2021年  【88页】AI新药研发（AIDD）行业系列报告：洞鉴行业发展，把握投资先机：（一）AIDD概览篇-AI新药研发已走过0到1阶段，1到10阶段将带来更多可能.pdf', 'format': 'PDF 1.5', 'keywords': '', 'modDate': "D:20210903094738+08'00'", 'page': 13, 'producer': 'Microsoft® Word 2010; modified using iText® 5.4.2 ©2000-2012 1T3XT BVBA (AGPL-version); modified using iTextSharp 5.2.1 (c) 1T3XT BVBA', 'source': 'source_documents/2021年  【88页】AI新药研发（AIDD）行业系列报告：洞鉴行业发展，把握投资先机：（一）AIDD概览篇-AI新药研发已走过0到1阶段，1到10

In [9]:
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from langchain import HuggingFacePipeline
embeddings = HuggingFaceEmbeddings(model_name=embeddings_model_name)
model = HuggingFacePipeline.from_model_id(model_id="/home/linjw/AGI/Weight/chinese-alpaca-2-13b-16k",
        task="text-generation",
        device=0,
        pipeline_kwargs={
            "max_new_tokens": 4096, 
            "do_sample": True,
            "temperature": 0.2,
            "top_k": 40,
            "top_p": 0.9,
            "repetition_penalty": 1.1},
        model_kwargs={
            "torch_dtype": "auto",
            "low_cpu_mem_usage": True}
        )

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:362: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/generation/configuration_utils.py:367: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


##### 定义目标模板

In [10]:
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

prompt_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "{context}\n{question} [/INST]"
)

refine_prompt_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "这是原始问题: {question}\n"
    "已有的回答: {existing_answer}\n"
    "现在还有一些文字，（如果有需要）你可以根据它们完善现有的回答。"
    "\n\n"
    "{context_str}\n"
    "\n\n"
    "请根据新的文段，进一步完善你的回答。"
    " [/INST]"
)

initial_qa_template = (
    "[INST] <<SYS>>\n"
    "You are a helpful assistant. 你是一个乐于助人的助手。\n"
    "<</SYS>>\n\n"
    "以下为背景知识：\n"
    "{context_str}"
    "\n"
    "请根据以上背景知识, 回答这个问题：{question}。"
    " [/INST]"
)


In [25]:
refine_prompt = PromptTemplate(
    input_variables=["question", "existing_answer", "context_str"],
    template=refine_prompt_template,
)
initial_qa_prompt = PromptTemplate(
    input_variables=["context_str", "question"],
    template=initial_qa_template,
)
chain_type_kwargs = {"question_prompt": initial_qa_prompt, "refine_prompt": refine_prompt}
qa = RetrievalQA.from_chain_type(
    llm=model, chain_type="refine",
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type_kwargs=chain_type_kwargs)


In [24]:
db.as_retriever(search_kwargs={"k": 10}).get_relevant_documents(query)

[Document(page_content='5\n1.1 行业定义\n医疗AI以助力降本增效、提升诊疗水平等为核\n心目的，全面赋能院前、院中、院后各个环节\n•\n医疗AI是指将计算机视觉、语音识别、自然语言处理、机器学习等人工智能技\n术应用于医疗领域中。近年来，随着人工智能技术的加速成熟，其在医疗健康\n领域的应用场景不断丰富，为疾病检测、诊断及治疗模式带来深刻变革，为提\n升居民健康质量提供新方式。\n•\n目前，人工智能技术在医疗领域的应用主要包括医学影像、临床辅助决策、精\n准医疗、健康管理、医疗信息化、药物研发以及医疗机器人等，以助力降本增\n效、提升诊疗水平、改善患者体验、降低患病风险等为核心目的，全面赋能院\n前、院中、院后各个环节。\n4\n图示：医疗AI主要应用场景及应用价值\n资料来源：36氪研究院根据公开资料整理\n场景\n主要细分类别\nAI技术应用价值\n医学影像\n疾病筛查\n提高筛查效率，降低误诊/漏诊率\n辅助诊断\n提高诊断准确性，实现疾病精准分级、分期\n临床辅助决策\n辅助治疗\n治疗方案辅助决策、术前规划、手术导航和预后评估等\n精准医疗\n基因检测\n提升检测效率及准确率，更好辅助疾病治疗\n健康管理\n可穿戴设备\n实现健康状况实时监测和评估', metadata={'author': '', 'creationDate': "D:20211221101338+02'13'", 'creator': 'WPS 演示', 'file_path': 'source_documents/2021年  【40页】36Kr-2021年中国医疗AI行业研究报告.pdf', 'format': 'PDF 1.7', 'keywords': '', 'modDate': "D:20211221101338+02'13'", 'page': 4, 'producer': '', 'source': 'source_documents/2021年  【40页】36Kr-2021年中国医疗AI行业研究报告.pdf', 'subject': '', 'title': '', 'total_pages': 40, 'trapped': ''}),
 Document(page_content='推想医疗：致力于打造全院级AI国际领军品牌\n

In [26]:
query = "医疗AI是什么"
qa.run(query)

/home/linjw/anaconda3/envs/llama/lib/python3.10/site-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


' 推想医疗是一家致力于利用人工智能深度学习技术开发多种平台，以满足不同医学需求的公司。其中，其肺结节检测与诊断是一项重要的功能之一。该平台名为InferRead CT，旨在提高肺结节的准确性检查和诊断水平。\n\nInferRead CT获得了国家药品监督管理局（NMPA）、欧盟医疗器械认证机构（CE）以及美国食品和药物管理局（FDA）的认证。这意味着它具备了高度可靠的质量标准和安全性要求。\n\nInferRead CT具有高效阅片的能力，能够迅速处理大量图像资料，并且具有较高的灵敏度。此外，它还提供了智能随访和疗效评估的功能。通过分析患者的历史影像，系统可以自动发现病变的变化情况，并帮助医生判断病情的发展趋势。\n\nInferRead CT还能够生成结构化报告，遵循权威指南的要求，包括肺结节数量、受影响肺叶的数量和程度等方面的信息。这些信息可以帮助医生更好地了解患者的情况，从而做出更加准确的诊断和治疗建议。\n\n总而言之，InferRead CT作为一种基于AI的肺结节检测与诊断工具，能够大大提高肺癌和其他肺部疾病的筛查效率和准确性，为临床实践带来了巨大的便利。'

In [27]:
qa_chain_refine = RetrievalQA.from_chain_type(
    model,
    retriever=db.as_retriever(search_kwargs={"k": 10}),
    chain_type="refine"
)
 
question = "医疗AI是什么"
result = qa_chain_refine({"query": question})
result["result"]

'\n\nRevised Answer:\n医疗AI是指将人工智能技术应用于临床诊疗各个环节中的医疗活动，旨在通过使用计算机视觉、语音识别、自然语言处理和机器学习等技术来协助医生进行准确的疾病检测、诊断和治疗，从而提升医疗服务质量和效率。例如，推想医疗是一家致力于利用AI深度学习技术开发多个平台，涵盖肺部、心脑血管、乳腺、骨关节等多个部位多脏器重大疾病，为客户提供“筛、诊、治、管、研”医疗全流程智慧解决方案的公司。其中，推想医疗的产品InferRead CT Lung和InferRead CT Pneumonia是针对肺部疾病的AI解决方案，能够提高肺癌筛查和肺炎诊断的准确性。这些产品的认证证明了其安全性和有效性，如NMPA、CE、FDA认证。\n\n此外，腾讯和百度也在医疗领域进行了布局。腾讯推出了首款将人工智能技术运用在医学领域的 AI 产品 —— “腾讯觅影”，它结合了图像识别、大数据处理、深度学习等领先的技术与医学跨界融合研发而成。腾讯觅影的核心功能包括 AI 医学影像和 AI 辅助诊疗，前者可以辅助医生诊断和重大疾病早期筛查；后者则依托于腾讯 AI Lab 的技术能力，通过自然语言处理和深度学习，为医生提供了更好的决策基础，帮助他们更快、更有效地理解病历并提升诊疗效率。\n\n百度也积极探索医疗AI的应用。2016年的魏则西事件促使百度开始重视医疗健康业务，并在此之后推出一系列相关产品和服务，如智能问诊、在线挂号、电子病历管理系统等。同时，百度还与其他医疗机构合作开展研究项目，以推动医疗AI的发展。'

In [28]:
result

{'query': '医疗AI是什么',
 'result': '\n\nRevised Answer:\n医疗AI是指将人工智能技术应用于临床诊疗各个环节中的医疗活动，旨在通过使用计算机视觉、语音识别、自然语言处理和机器学习等技术来协助医生进行准确的疾病检测、诊断和治疗，从而提升医疗服务质量和效率。例如，推想医疗是一家致力于利用AI深度学习技术开发多个平台，涵盖肺部、心脑血管、乳腺、骨关节等多个部位多脏器重大疾病，为客户提供“筛、诊、治、管、研”医疗全流程智慧解决方案的公司。其中，推想医疗的产品InferRead CT Lung和InferRead CT Pneumonia是针对肺部疾病的AI解决方案，能够提高肺癌筛查和肺炎诊断的准确性。这些产品的认证证明了其安全性和有效性，如NMPA、CE、FDA认证。\n\n此外，腾讯和百度也在医疗领域进行了布局。腾讯推出了首款将人工智能技术运用在医学领域的 AI 产品 —— “腾讯觅影”，它结合了图像识别、大数据处理、深度学习等领先的技术与医学跨界融合研发而成。腾讯觅影的核心功能包括 AI 医学影像和 AI 辅助诊疗，前者可以辅助医生诊断和重大疾病早期筛查；后者则依托于腾讯 AI Lab 的技术能力，通过自然语言处理和深度学习，为医生提供了更好的决策基础，帮助他们更快、更有效地理解病历并提升诊疗效率。\n\n百度也积极探索医疗AI的应用。2016年的魏则西事件促使百度开始重视医疗健康业务，并在此之后推出一系列相关产品和服务，如智能问诊、在线挂号、电子病历管理系统等。同时，百度还与其他医疗机构合作开展研究项目，以推动医疗AI的发展。'}